In [1]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict

In [2]:
def getCDF(data):
    xdata = np.sort(data)
    ydata = [i/len(xdata) for i in range(len(xdata))]
    return xdata, ydata

In [3]:
DNS_lookups = {
"Nest Thermostat": ["transport.home.nest.com", "frontdoor.nest.com.", "time.nest.com.", "weather.nest.com.", "devices.nest.com."],
"SmartThings": ["pool.ntp.org.", "DC.connect.smartthings.com."],
"Pixstar Photoframe": ["api.pix-star.com.", "spheris.connectify.", "iptime.pix-star.com."],
"Nest Cam": ["nexus.dropcam.com.", "oculus*.dropcam.com.", "files.dropcam.com.", "pool.ntp.org."],
"Amazon Echo": ["amzdigitaldownloads.edgesuite.net.", "softwareupdates.amazon.com.", "todo-*.amazon.com.", "ntp.amazon.com.", "pindorama.amazon.com.",  "firs-*.amazon.com.", "ntp-*.amazon.com.", "*.spotify.com.", "guipitan.amazon.com.", "*.north-america.pool.ntp.org.", "spectrum.s3.amazonaws.com.", "pins.amazon.com.", "device-metrics-us.amazon.com."],
"Sharx IPCam": ["www.sharxsecurity.com.", "time.nist.gov."],
"Ubi": ["clients3.google.com.", "api.grooveshark.com.", "dl.google.com.", "www.google.ca.", "*.grooveshark.com.", "www.earthtools.org.", "play.googleapis.com.", "ip-api.com.", "android.clients.google.com.", "mtalk.google.com.", "portal.theubi.com.", "*.android.pool.ntp.org.", "www.google.com."]
}

# OTHER INTERESTING DNS: https://www.nanog.org/sites/default/files/Nadji.pdf

In [4]:
DNS_moderate_lookups = {
    'nest_therm': '.nest.com',
    'nest_cam': '.dropcam.com',
    'smartthings': '.smartthings.com',
    'pixstar': '.pix-star.com',
    'echo': 'device-metrics-us.amazon.com',
    'sharx': 'sharxsecurity.com',
    'august': '.august.com',
    'hue': '.meethue.com',
    #'apple': 'time-ios.apple.com',
    'apple': 'gs-loc.ls-apple.com',
    'ps': '.playstation.net',
    'xbox': 'xboxlive.com',
    'nintendo': 'nintendo.net',
}

DNS_strict_lookups = {
    'nest_therm': 'devices.nest.com',
    'nest_cam': 'nexus.dropcam.com',
    'smartthings': 'api.smartthings.com',
    'pixstar': 'api.pix-star.com',
    'echo': 'device-metrics-us.amazon.com',
    'sharx': 'sharxsecurity.com',
    'august': 'doorbells.august.com',
    'hue': 'bridge.meethue.com',
    'apple': 'gs-loc.ls-apple.com',
    'ps': 'update.playstation.net',
    'xbox': '.xboxlive.com',
    'nintendo': '.nintendo.net',
}

In [5]:
df = pd.read_csv("pima-extract.csv", sep="|").fillna('')

In [6]:
df.head()

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
0,0.000000,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,24.9.76.250,69.252.212.78,android.clients.google.com
1,0.000001,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.82,69.31.113.189,e3645.b.akamaiedge.net
2,0.000005,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,24.9.76.250,69.252.212.88,android.clients.google.com
3,0.000009,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,24.9.76.250,69.252.212.84,android.clients.google.com
4,0.000010,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,73.153.66.183,69.252.212.76,www.polls.newsvine.com


# STATS

In [7]:
print "Total time of dns record [677 MB / Comcast-PIMA]:", (df.iloc[-1]['frame.time_relative'] - df.iloc[0]['frame.time_relative']), "s"
print "Total number of queries: ", len(df)
print "Queries without source IP (are probably IPv6 so IP was not extracted): ", len(df[df['ip.src']==''])

print "Unique source IPs (homes):", len(df['ip.src'].unique())
print "Unique destination IPs (DNS servers):", len(df['ip.dst'].unique())

print "Unique queries:", len(df['dns.qry.name'].unique())

Total time of dns record [677 MB / Comcast-PIMA]: 59.990207 s
Total number of queries:  2527293
Queries without source IP (are probably IPv6 so IP was not extracted):  502648
Unique source IPs (homes): 260841
Unique destination IPs (DNS servers): 11767
Unique queries: 129065


# To Do
- Why are there 11767 unique servers?
- Hypothesis: home-dns popularity based on threshold to filter out DNS
    - there will not be many homes with IoT related DNS [use second level domain (*.iot.com)]
    - homes with IoTs will have similar queries (individual queries not in other IoT homes can be removed)
    - each IoT device will have at least 1 regular (heartbeat) IoT query (not *.iot.com)
- Tree
- Related Work

# MAIN: Interesting Questions
- A way to blacklist popular non-IoT related domains
- Abnormal/malicious domain queries check
- Time based profile per device DNS query [ srcIP | domain | mean (T) | var (T) ]

## Time profile of DNS queries per IP

In [8]:
gp = df.groupby(['ip.src', 'dns.qry.name'])

In [9]:
qry_time_mean = gp['frame.time_relative'].mean()
qry_time_std = gp['frame.time_relative'].std()
qry_count = gp['frame.time_relative'].count()

In [10]:
qry_time_mean

ip.src         dns.qry.name                                                                                                                                            
                f5.sensr.net                                                                                                                                               29.860547
               $PROXY.utopia.net                                                                                                                                           33.304035
               %20http.hsd1.co.comcast.net                                                                                                                                 54.459351
               %24staticdomain                                                                                                                                             31.938133
               %24staticdomain.hsd1.co.comcast.net                                                          

## Time profile of known IoT based DNS queries per IP
- many IoTs (like Amazon Echo) seem to be using IPv6. Current trace only extracted IPv4 so limit analysis to valid source IPs for now

In [11]:
iot_list = defaultdict(int)
for domain in DNS_moderate_lookups.values():
    if len(iot_list) == 0:
        iot_list = df['dns.qry.name'].str.contains(domain)
    else:
        iot_list = iot_list | df['dns.qry.name'].str.contains(domain)
df_iot = df[iot_list]

In [12]:
df_iot.head()

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
156,0.425989,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,173.14.26.110,69.252.212.80,fus01.ps4.update.playstation.net
169,0.426302,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,71.229.198.23,69.252.212.84,log-rts11-iad01.devices.nest.com
290,0.428878,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,69.244.12.6,69.252.212.78,www.meethue.com
369,0.430517,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,24.9.133.179,69.252.212.74,www.sharxsecurity.com
822,0.441398,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,67.177.225.0,69.252.212.78,licensing.xboxlive.com


### Add corresponding device column based on query

In [13]:
inverse_DNS_lookups = {v: k for k, v in DNS_moderate_lookups.items()}
df_iot['device'] = ''

for domain in DNS_moderate_lookups.values():
    df_iot.loc[ df_iot['dns.qry.name'].str.contains(domain), 'device'] = inverse_DNS_lookups[domain]

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
df_iot.head(10)

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name,device
156,0.425989,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,173.14.26.110,69.252.212.80,fus01.ps4.update.playstation.net,ps
169,0.426302,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,71.229.198.23,69.252.212.84,log-rts11-iad01.devices.nest.com,nest_therm
290,0.428878,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,69.244.12.6,69.252.212.78,www.meethue.com,hue
369,0.430517,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,24.9.133.179,69.252.212.74,www.sharxsecurity.com,sharx
822,0.441398,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,67.177.225.0,69.252.212.78,licensing.xboxlive.com,xbox
881,0.442581,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,,,sessiondirectory.xboxlive.com,xbox
882,0.442584,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,,,sessiondirectory.xboxlive.com,xbox
948,0.444548,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,174.51.70.150,69.252.212.88,gs-loc.ls-apple.com.akadns.net,apple
1058,0.446752,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,71.229.163.73,69.252.212.88,static-resource.np.community.playstation.net,ps
1065,0.446888,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,,,xsts.auth.xboxlive.com,xbox


### Extract only IPv4-DNS

In [15]:
df_iot_ipv4 = df_iot[df_iot['ip.src']!='']
print "Known IoT queries: ", len(df_iot)
print "Known IoT queries with valid IPv4: ", len(df_iot_ipv4)

Known IoT queries:  18316
Known IoT queries with valid IPv4:  11863


In [16]:
print "List of unique IoT domains extracted (IPv4 only):",
df_iot_ipv4['dns.qry.name'].unique()

List of unique IoT domains extracted (IPv4 only):

array(['fus01.ps4.update.playstation.net',
       'log-rts11-iad01.devices.nest.com', 'www.meethue.com',
       'www.sharxsecurity.com', 'licensing.xboxlive.com',
       'gs-loc.ls-apple.com.akadns.net',
       'static-resource.np.community.playstation.net',
       'data-vef.xboxlive.com', 'data-vef.gtm.xboxlive.com',
       'us-prof.np.community.playstation.net', 'xsts.auth.xboxlive.com',
       'sessiondirectory.xboxlive.com', 'asm.np.community.playstation.net',
       'device-metrics-us.amazon.com', 'nsx.np.dl.playstation.net',
       'log-rts02-iad01.devices.nest.com',
       'gs2.ww.prod.dl.playstation.net', 'title.mgt.xboxlive.com',
       'image.xboxlive.com.nsatc.net', 'social.gtm.xboxlive.com',
       'avatarservices.gtm.xboxlive.com',
       'ps4-system.sec.np.dl.playstation.net',
       'transport01-rts09-iad01.transport.home.nest.com',
       'event.api.np.km.playstation.net', 'settings.xboxlive.com',
       'olvus.cdn.nintendo.net', 'msg.xboxlive.com',
       'msg.gtm.xbox

In [18]:
# SAVE IOT DNS DATAFRAME
df_iot_ipv4.to_pickle('processed/df_iot_device_ipv4.pkl')

In [167]:
gp = df_iot_ipv4.groupby(['ip.src', 'dns.qry.name', 'device'])

In [178]:
iot_count = gp['frame.time_relative'].count()

In [191]:
iot_count.sort(ascending=False)
iot_count.head(10)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting
  if __name__ == '__main__':


ip.src         dns.qry.name                         device     
69.244.4.108   graph.api.smartthings.com            smartthings    123
75.70.226.15   gameclipscontent-d3008.xboxlive.com  xbox            27
               gameclipscontent-d2013.xboxlive.com  xbox            14
73.24.144.121  xsts.auth.xboxlive.com               xbox            13
71.205.176.37  cs.prd.msys.playstation.net          ps              11
73.95.103.156  device.auth.xboxlive.com             xbox            11
75.70.226.15   avatarservices.xboxlive.com          xbox            11
70.90.112.54   log-rts18-iad01.devices.nest.com     nest_therm      10
73.24.144.121  profile.xboxlive.com                 xbox            10
73.95.103.156  title.mgt.xboxlive.com               xbox             9
Name: frame.time_relative, dtype: int64

In [180]:
iot_mean_t = gp['frame.time_relative'].mean()
iot_std_t = gp['frame.time_relative'].std()

In [188]:
iot_mean_t.head()

ip.src        dns.qry.name                    device    
107.0.87.50   home.nest.com                   nest_therm     8.279527
              oculus877-vir.dropcam.com       nest_cam      38.292396
107.1.83.162  gs-loc.ls-apple.com.akadns.net  apple         57.600348
107.2.0.212   assets1.xboxlive.com            xbox          13.591052
              gs2.ww.prod.dl.playstation.net  ps            11.417153
Name: frame.time_relative, dtype: float64

In [189]:
iot_std_t.head()

ip.src        dns.qry.name                    device    
107.0.87.50   home.nest.com                   nest_therm         NaN
              oculus877-vir.dropcam.com       nest_cam           NaN
107.1.83.162  gs-loc.ls-apple.com.akadns.net  apple              NaN
107.2.0.212   assets1.xboxlive.com            xbox          0.000003
              gs2.ww.prod.dl.playstation.net  ps                 NaN
Name: frame.time_relative, dtype: float64

### Combine IoT dataframe as [ip, dns, device, count, mean, std]

In [196]:
iot_summary = pd.concat({'count': iot_count, 'mean': iot_mean_t, 'std': iot_std_t}, axis=1).reset_index()

#### Device vs number of queries
- Overall most queries: xbox, ps, echo, apple, nest_therm, nintendo, smartthings, nest_cam, hue, august, pixstar, sharx
- average queries per IP for each device

In [204]:
iot_summary.groupby('device')['count'].sum()

device
apple          1342
august           30
echo           2286
hue              39
nest_cam         82
nest_therm      999
nintendo        608
pixstar          22
ps             2981
sharx             7
smartthings     190
xbox           3277
Name: count, dtype: int64

#### IP vs Number of Queries

In [212]:
g1 = iot_summary.groupby(['ip.src', 'device'])['count'].sum().reset_index()
g2 = g1.pivot(index='ip.src', columns='device', values='count').fillna(0)
g2

device,apple,august,echo,hue,nest_cam,nest_therm,nintendo,pixstar,ps,sharx,smartthings,xbox
ip.src,,,,,,,,,,,,
107.0.87.50,0,0,0,0,1,1,0,0,0,0,0,0
107.1.83.162,1,0,0,0,0,0,0,0,0,0,0,0
107.2.0.212,0,0,0,0,0,0,0,0,1,0,0,2
107.2.1.145,2,0,0,0,0,0,0,0,0,0,0,0
107.2.1.64,0,0,0,0,0,0,0,0,0,0,0,1
107.2.1.73,2,0,0,0,0,0,0,0,0,0,0,0
107.2.10.205,1,0,0,0,0,0,0,0,0,0,0,0
107.2.11.123,0,0,0,0,0,0,0,0,1,0,0,0
107.2.12.105,0,0,1,0,0,0,0,0,0,0,0,0


#### Average number of queries per IP for each device (biased)

In [221]:
g2.mean()

device
apple          0.208159
august         0.004653
echo           0.354584
hue            0.006049
nest_cam       0.012719
nest_therm     0.154956
nintendo       0.094307
pixstar        0.003412
ps             0.462386
sharx          0.001086
smartthings    0.029471
xbox           0.508298
dtype: float64

#### Average number of queries per IP for each device for only those IPs with the device (unbiased)
- most queries from devices: sharx, smartthings, xbox, ps, august, nest_therm, nintendo, nest_cam, pixstar, hue, apple, echo

In [220]:
g1.groupby('device')['count'].mean()

device
apple          1.223336
august         2.307692
echo           1.118395
hue            1.300000
nest_cam       1.782609
nest_therm     2.134615
nintendo       1.993443
pixstar        1.692308
ps             2.409863
sharx          3.500000
smartthings    3.015873
xbox           2.471342
Name: count, dtype: float64

### Remove IPs with query count = 1
- mean, std of time only makes sense for repeated queries

In [199]:
iot_many_query = iot_summary[iot_summary['count']>1]

print "Total IoT related queries: ", len(df_iot_ipv4)
print "Unique IoT related IP-DNS combinations: ", len(iot_summary)
print "Unique IoT related IP-DNS combinations with only 1 query: ", len(iot_summary[iot_summary['count']==1])
print "Unique IoT related IP-DNS combinations with more than 1 query: ", len(iot_many_query)

Total IoT related queries:  11863
Unique IoT related IP-DNS combinations:  9736
Unique IoT related IP-DNS combinations with only 1 query:  8246
Unique IoT related IP-DNS combinations with more than 1 query:  1490


In [222]:
iot_many_query

,ip.src,dns.qry.name,device,count,mean,std
3,107.2.0.212,assets1.xboxlive.com,xbox,2,13.591052,2.828427e-06
5,107.2.1.145,gs-loc.ls-apple.com.akadns.net,apple,2,44.003446,2.824892e-03
7,107.2.1.73,gs-loc.ls-apple.com.akadns.net,apple,2,8.277703,5.374012e-05
14,107.2.13.182,licensing.xboxlive.com,xbox,2,24.020494,4.242641e-06
31,107.2.135.94,xflight.xboxlive.com,xbox,2,45.735158,2.932018e-02
36,107.2.138.90,transport01.rts18.iad01.production.nest.com,nest_therm,4,46.240981,1.451810e+00
44,107.2.144.55,bravia.dl.playstation.net,ps,2,44.176015,3.288903e+00
48,107.2.150.198,gs-loc.ls-apple.com.akadns.net,apple,2,28.628558,3.392698e-03
49,107.2.150.222,update-cdn.xboxlive.com,xbox,2,19.854548,2.068509e+00
50,107.2.151.241,doorbells.august.com,august,4,12.463299,1.389989e+01


In [195]:
iot_summary

count  \
ip.src        dns.qry.name                                       device              
107.0.87.50   home.nest.com                                      nest_therm      1   
              oculus877-vir.dropcam.com                          nest_cam        1   
107.1.83.162  gs-loc.ls-apple.com.akadns.net                     apple           1   
107.2.0.212   assets1.xboxlive.com                               xbox            2   
              gs2.ww.prod.dl.playstation.net                     ps              1   
107.2.1.145   gs-loc.ls-apple.com.akadns.net                     apple           2   
107.2.1.64    update-cdn.xboxlive.com                            xbox            1   
107.2.1.73    gs-loc.ls-apple.com.akadns.net                     apple           2   
107.2.10.205  gs-loc.ls-apple.com.akadns.net                     apple           1   
107.2.11.123  bravia-e.dl.playstation.net                        ps              1   
107.2.12.105  device-metrics-us.amazon.com                       echo            1   
107.2.12.8    device-metrics-us.amazon.com                       echo            1   
107.2.128.43  gs-loc.ls-apple.com.akadns.net                     apple           1   
107.2.13.162  device-metrics-us.amazon.com                       echo            1   
107.2.13.182  licensing.xboxlive.com                             xbox            2   
107.2.131.149 device-metrics-us.amazon.com                       echo            1   
107.2.134.125 eplists.xboxlive.com                               xbox            1   
              www.xboxlive.com                                   xbox            1   
107.2.135.104 device-metrics-us.amazon.com                       echo            1   
              gs-loc.ls-apple.com.akadns.net                     apple           1   
107.2.135.240 attestation.xboxlive.com                           xbox            1   
              title.mgt.xboxlive.com                             xbox            1   
              update-cdn.xboxlive.com                            xbox            1   
              update.xboxlive.com                                xbox            1   
              xsts.auth.xboxlive.com                             xbox            1   
107.2.135.86  52-40-147-158-push.np.communication.playstation... ps              1   
              asm.np.community.playstation.net                   ps              1   
              auth.api.np.ac.playstation.net                     ps              1   
              fus01.ps4.update.playstation.net                   ps              1   
              us-push.np.communication.playstation.net           ps              1   
...                                                                            ...   
98.245.74.177 device-metrics-us.amazon.com                       echo            1   
98.245.74.215 npdi.cdn.nintendo.net                              nintendo        1   
98.245.74.241 gs-loc.ls-apple.com.akadns.net                     apple           1   
98.245.77.113 gs-loc.ls-apple.com.akadns.net                     apple           2   
98.245.78.15  gs-loc.ls-apple.com.akadns.net                     apple           1   
98.245.79.59  device-metrics-us.amazon.com                       echo            1   
98.245.8.238  gs2.ww.prod.dl.playstation.net                     ps              1   
98.245.80.203 update-cdn.xboxlive.com                            xbox            1   
98.245.82.238 nppl.app.nintendo.net                              nintendo        1   
98.245.82.61  us-prof.np.community.playstation.net               ps              1   
98.245.83.226 gs2.ww.prod.dl.playstation.net                     ps              1   
98.245.83.86  ps4-system.sec.np.dl.playstation.net               ps              1   
98.245.84.125 us-prof.np.community.playstation.net               ps              1   
98.245.85.64  us-prof.np.community.playstation.net               ps              1   
98.245.87.30  nppl.app.nintendo.net

# Extract known IoT-DNS by src IP

In [25]:
df_IP = pd.DataFrame({'ip.src': df['ip.src'].unique()}).sort_values(by='ip.src').set_index('ip.src')

In [27]:
ip_dns = defaultdict(int)
for domain in DNS_moderate_lookups.values():
    #ip_dns[domain] = df['dns.qry.name'].str.contains(domain)
    temp = pd.DataFrame( { 'ip.src' : df[ df['dns.qry.name'].str.contains(domain) ]['ip.src'].unique(),
              domain: True} ).set_index('ip.src')
    df_IP[domain]=temp
    df_IP[domain].fillna(False, inplace=True)

In [28]:
df_IP.head()

,.meethue.com,gs-loc.ls-apple.com,device-metrics-us.amazon.com,sharxsecurity.com,nintendo.net,.dropcam.com,.august.com,.playstation.net,.smartthings.com,.pix-star.com,xboxlive.com,.nest.com
ip.src,,,,,,,,,,,,
,True,True,True,False,True,True,True,True,True,False,True,True
106.184.1.178,False,False,False,False,False,False,False,False,False,False,False,False
107.0.141.78,False,False,False,False,False,False,False,False,False,False,False,False
107.0.143.131,False,False,False,False,False,False,False,False,False,False,False,False
107.0.164.246,False,False,False,False,False,False,False,False,False,False,False,False


### CHECK particular IP

In [29]:
df_IP.ix['98.225.66.206']

.meethue.com                    False
gs-loc.ls-apple.com             False
device-metrics-us.amazon.com    False
sharxsecurity.com               False
nintendo.net                    False
.dropcam.com                    False
.august.com                     False
.playstation.net                False
.smartthings.com                False
.pix-star.com                   False
xboxlive.com                    False
.nest.com                       False
Name: 98.225.66.206, dtype: bool

### ANALYSIS

In [30]:
df_IP.sum()

.meethue.com                      31
gs-loc.ls-apple.com             1098
device-metrics-us.amazon.com    2045
sharxsecurity.com                  2
nintendo.net                     306
.dropcam.com                      47
.august.com                       14
.playstation.net                1238
.smartthings.com                  64
.pix-star.com                     13
xboxlive.com                    1327
.nest.com                        469
dtype: int64

#### sort source IPs by total number of DNS-IoT found in 1 min trace

Actual PIMA src IPs with IoTs = 17577 (current trace union another trace) from around 250k

In [34]:
indx = df_IP.sum(axis=1).sort_values(ascending=False).index

In [35]:
df_IP_sorted = df_IP.reindex(indx)
df_IP_sorted['total'] = df_IP_sorted.sum(axis=1)

In [40]:
print "Source IPs with IoTs:", len(df_IP_sorted[ df_IP_sorted.total>0 ])
print "Source IPs without IoTs:", len(df_IP_sorted[ df_IP_sorted.total==0 ])
print "Percentage of IoT source IPs in current trace: ", (len(df_IP_sorted[ df_IP_sorted.total>0 ])/len(df_IP_sorted[ df_IP_sorted.total==0 ]) * 100)

Source IPs with IoTs: 6448
Source IPs without IoTs: 254393
Percentage of IoT source IPs in current trace:  2.534660938


In [43]:
# Distribution of number of IoTs per home
tot = df_IP_sorted[ df_IP_sorted.total>0 ]['total']
tot.head(10)

ip.src
69.241.95.83     11
                 10
69.241.95.85     10
69.241.95.82     10
69.241.95.87      9
69.241.95.86      9
69.241.95.84      9
73.95.130.62      5
73.95.130.127     5
73.95.130.28      5
Name: total, dtype: int64

In [44]:
df_IP_sorted.to_pickle("pima-extract-sorted-IoT-IPs.pkl")

In [45]:
from collections import Counter
print "Distribution of number of IoTs per home: ", Counter(tot)

Distribution of number of IoTs per home:  Counter({1: 6345, 2: 72, 4: 11, 5: 7, 3: 6, 9: 3, 10: 3, 11: 1})


In [46]:
interesting = df_IP_sorted[df_IP_sorted.total > 2]
print len(interesting)
interesting.sum().sort_values(ascending=False)

31


total                           165
xboxlive.com                     31
.playstation.net                 30
device-metrics-us.amazon.com     29
gs-loc.ls-apple.com              24
nintendo.net                     15
.nest.com                        10
.august.com                       7
.dropcam.com                      7
.smartthings.com                  6
.pix-star.com                     4
.meethue.com                      2
sharxsecurity.com                 0
dtype: float64

- Most popular Xboxlive, Playstation, Amazon Echo
- Followed by Nintendo, Apple Watch, Nest thermostat
- August Lock, Dropcam, Smartthings


### SUMMARY 2: USING STRICT DOMAINS instead of MODERATE

In [47]:
df_IP2 = pd.DataFrame({'ip.src': df['ip.src'].unique()}).sort_values(by='ip.src').set_index('ip.src')

ip_dns2 = defaultdict(int)
for domain in DNS_strict_lookups.values():
    #ip_dns[domain] = df['dns.qry.name'].str.contains(domain)
    temp = pd.DataFrame( { 'ip.src' : df[ df['dns.qry.name'].str.contains(domain) ]['ip.src'].unique(),
              domain: True} ).set_index('ip.src')
    df_IP2[domain]=temp
    df_IP2[domain].fillna(False, inplace=True)

In [48]:
df_IP2.sum()

bridge.meethue.com                15
gs-loc.ls-apple.com             1098
device-metrics-us.amazon.com    2045
sharxsecurity.com                  2
.nintendo.net                    306
nexus.dropcam.com                 14
doorbells.august.com              13
update.playstation.net           197
api.smartthings.com                6
api.pix-star.com                  13
.xboxlive.com                   1327
devices.nest.com                 297
dtype: int64

### CHECK particular IP

In [49]:
df_IP2.ix['69.244.36.23']

bridge.meethue.com              False
gs-loc.ls-apple.com             False
device-metrics-us.amazon.com    False
sharxsecurity.com               False
.nintendo.net                   False
nexus.dropcam.com               False
doorbells.august.com            False
update.playstation.net          False
api.smartthings.com             False
api.pix-star.com                False
.xboxlive.com                   False
devices.nest.com                False
Name: 69.244.36.23, dtype: bool

In [50]:
#pd.concat( [ df['dns.qry.name'].str.contains(domain), df['ip.src'] ], axis=1 ).groupby('ip.src').any()

In [51]:
#df_ip_dns = pd.DataFrame(ip_dns).join(df['ip.src'])

In [52]:
#ip_dns = defaultdict(int)
#for domain in DNS_lookups.values():
#    ip_dns[domain] = df[ df['dns.qry.name'].str.contains(domain) ]['ip.src'].unique()

In [53]:
#ip_dns

In [54]:
#cols = list(df_ip_dns.columns)

In [55]:
#df_ip_dns

### NEST THERMOSTAT

In [56]:
nest_df = df[df['dns.qry.name'].str.contains("nest.com")]

nest_df

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
169,0.426302,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,71.229.198.23,69.252.212.84,log-rts11-iad01.devices.nest.com
3089,0.495322,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,73.95.47.13,69.252.212.68,log-rts02-iad01.devices.nest.com
3372,0.501875,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,73.95.47.13,69.252.212.68,log-rts02-iad01.devices.nest.com
5564,0.555964,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,,,transport01-rts09-iad01.transport.home.nest.com
5586,0.556278,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.86,205.251.195.86,transport01-rts09-iad01.transport.home.nest.com
9870,0.656782,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,174.51.73.184,69.252.212.72,log-rts20-iad01.devices.nest.com
17405,0.83489,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,,,log-rts11-iad01.devices.nest.com
20185,0.897977,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,71.229.148.200,69.252.212.76,frontdoor.nest.com
20192,0.898152,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.84,205.251.195.86,frontdoor.nest.com
22159,0.943734,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,,,transport04-rts08-iad01.transport.home.nest.com


## NEST CAM

In [57]:
nestcam_df = df[df['dns.qry.name'].str.contains("dropcam.com")]

nestcam_df

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
63693,1.94753,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,73.24.249.230,69.252.212.78,nexus.dropcam.com
101781,2.86947,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,73.153.148.4,69.252.212.84,www.dropcam.com
140529,3.79976,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,50.170.223.125,69.252.212.76,stream-alfa.dropcam.com
182371,4.77707,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,73.217.92.123,69.252.212.80,www.dropcam.com
182814,4.78651,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,75.70.245.39,69.252.212.72,nexusapi.dropcam.com
216806,5.59157,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,73.153.250.116,69.252.212.70,www.dropcam.com
216814,5.59175,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.83,205.251.194.9,www.dropcam.com
253130,6.43821,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,24.11.150.189,69.252.212.74,nexus.dropcam.com
253135,6.43844,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.84,205.251.194.9,nexus.dropcam.com
314078,7.89541,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,24.11.150.189,69.252.212.70,oculus121-vir.dropcam.com


## SMARTTHINGS

In [58]:
smartthings_df = df[df['dns.qry.name'].str.contains("smartthings.com")]

smartthings_df

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
35717,1.26441,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,69.244.4.108,69.252.212.78,graph.api.smartthings.com
36055,1.27235,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,69.244.4.108,69.252.212.70,graph.api.smartthings.com
77250,2.26884,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,69.244.4.108,69.252.212.78,graph.api.smartthings.com
77587,2.27695,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,69.244.4.108,69.252.212.80,graph.api.smartthings.com
97418,2.76591,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,107.2.8.57,69.252.212.82,fw-update.smartthings.com
97431,2.76615,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.86,205.251.195.11,fw-update.smartthings.com
118686,3.27225,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,69.244.4.108,69.252.212.86,graph.api.smartthings.com
119201,3.28495,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,69.244.4.108,69.252.212.72,graph.api.smartthings.com
161318,4.27683,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,69.244.4.108,69.252.212.76,graph.api.smartthings.com
162910,4.31061,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,69.244.4.108,69.252.212.80,graph.api.smartthings.com


## PIX-STAR

In [59]:
pixstar_df = df[df['dns.qry.name'].str.contains("pix-star.com")]

pixstar_df

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
95755,2.72294,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,75.70.222.3,69.252.212.74,api.pix-star.com
256828,6.528,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,71.205.156.203,69.252.212.80,api.pix-star.com
339901,8.50164,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,24.11.158.28,69.252.212.66,api.pix-star.com
339911,8.50186,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.82,84.233.158.131,api.pix-star.com
346779,8.66756,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,24.11.158.28,69.252.212.70,api.pix-star.com.hsd1.az.comcast.net
347916,8.6936,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,24.11.158.28,69.252.212.78,api.pix-star.com.#
348634,8.71155,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,24.11.158.28,69.252.212.88,api.pix-star.com.wlan0
348649,8.71172,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.87,199.7.91.13,api.pix-star.com.wlan0
350370,8.75358,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,24.11.158.28,69.252.212.66,api.pix-star.com
578200,14.197,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,71.237.68.136,69.252.212.74,api.pix-star.com


## AMAZON ECHO

In [60]:
echo_df = df[df['dns.qry.name'].str.contains("device-metrics-us.amazon.com")]

echo_df


,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
2211,0.47402,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,,,device-metrics-us.amazon.com
2682,0.485964,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,,,device-metrics-us.amazon.com
2793,0.488358,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,,,device-metrics-us.amazon.com
3020,0.493752,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,69.244.44.19,69.252.212.80,device-metrics-us.amazon.com
3490,0.504294,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,,,device-metrics-us.amazon.com
4479,0.529533,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,50.170.175.217,69.252.212.80,device-metrics-us.amazon.com
4864,0.53921,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,,,device-metrics-us.amazon.com
6334,0.57313,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,,,device-metrics-us.amazon.com
6629,0.579893,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,,,device-metrics-us.amazon.com
6740,0.582915,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,,,device-metrics-us.amazon.com


## SHARX IPCAM

In [61]:
sharx_df = df[df['dns.qry.name'].str.contains("sharxsecurity.com")]

sharx_df

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
369,0.430517,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,24.9.133.179,69.252.212.74,www.sharxsecurity.com
556091,13.6497,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,24.9.133.179,69.252.212.82,www.sharxsecurity.com
853647,20.7922,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,24.9.133.179,69.252.212.70,www.sharxsecurity.com
1218129,29.4658,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,73.95.22.25,69.252.212.74,www.sharxsecurity.com
1639121,39.3657,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,24.9.133.179,69.252.212.78,www.sharxsecurity.com
1826859,43.6617,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,24.9.133.179,69.252.212.84,www.sharxsecurity.com
2130890,50.8498,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,24.9.133.179,69.252.212.70,www.sharxsecurity.com


##  UBI

In [62]:
ubi_df = df[df['dns.qry.name'].str.contains("theubi.com")]

ubi_df


,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name


## AUGUST LOCK

In [63]:
august_df = df[df['dns.qry.name'].str.contains("august.com")]

august_df


,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
56343,1.76712,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,107.2.151.241,69.252.212.80,doorbells.august.com
56349,1.76737,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.85,205.251.195.2,doorbells.august.com
77456,2.27393,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,174.51.52.254,69.252.212.80,doorbells.august.com
236160,6.03875,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,,,doorbells.august.com
301253,7.5919,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,107.2.151.241,69.252.212.88,doorbells.august.com
301254,7.5919,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,107.2.151.241,69.252.212.68,doorbells.august.com
333593,8.34986,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,174.51.52.254,69.252.212.80,doorbells.august.com
333805,8.35463,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,174.51.52.254,69.252.212.76,doorbells.august.com
335296,8.38933,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,174.51.52.254,69.252.212.78,doorbells.august.com.hsd1.co.comcast.net
383895,9.55984,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,67.190.56.235,69.252.212.74,doorbells.august.com


## PHILLIPS HUE

In [64]:
hue_df = df[df['dns.qry.name'].str.contains("meethue.com")]

hue_df


,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
290,0.428878,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,69.244.12.6,69.252.212.78,www.meethue.com
96133,2.73291,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,71.205.42.196,69.252.212.68,bridge.meethue.com
96141,2.73304,00:1c:73:69:93:01,e0:ac:f1:39:39:53,,,bridge.meethue.com
243352,6.20694,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,73.24.242.243,69.252.212.82,bridge.meethue.com
399630,9.92882,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,73.153.137.118,69.252.212.88,bridge.meethue.com
402097,9.98414,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,24.9.91.88,69.252.212.68,www.meethue.com
454705,11.2076,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,73.24.242.243,69.252.212.68,bridge.meethue.com
475484,11.7104,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,50.170.153.17,69.252.212.78,www.meethue.com
544799,13.3828,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,73.243.104.162,69.252.212.66,bridge.meethue.com
700825,17.1368,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,,,www.meethue.com


## APPLE WATCH

In [65]:
#apple_df = df[df['dns.qry.name'].str.contains("time-ios.g.aaplimg.com")]
apple_df = df[df['dns.qry.name'].str.contains("gs-loc.ls-apple.com")]

apple_df


,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
948,0.444548,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,174.51.70.150,69.252.212.88,gs-loc.ls-apple.com.akadns.net
1232,0.450624,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,71.56.206.100,69.252.212.68,gs-loc.ls-apple.com.akadns.net
2531,0.482263,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,73.14.175.66,69.252.212.82,gs-loc.ls-apple.com.akadns.net
5301,0.549262,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,75.70.134.240,69.252.212.72,gs-loc.ls-apple.com.akadns.net
9388,0.645707,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,71.56.197.192,69.252.212.68,gs-loc.ls-apple.com.akadns.net
9428,0.646887,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,68.84.206.219,69.252.212.88,gs-loc.ls-apple.com.akadns.net
10036,0.660586,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,,,gs-loc.ls-apple.com.akadns.net
10271,0.666278,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,71.237.3.228,69.252.212.72,gs-loc.ls-apple.com.akadns.net
11401,0.693539,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,,,gs-loc.ls-apple.com.akadns.net
12502,0.718924,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,,,gs-loc.ls-apple.com.akadns.net


## PLAYSTATION

In [66]:
ps_df = df[df['dns.qry.name'].str.contains("update.playstation.net")]

ps_df



,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
156,0.425989,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,173.14.26.110,69.252.212.80,fus01.ps4.update.playstation.net
23582,0.973793,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,24.9.1.156,69.252.212.86,fus01.ps4.update.playstation.net
24491,0.994319,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,69.244.36.65,69.252.212.76,fus01.ps4.update.playstation.net
29469,1.11212,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,73.217.46.201,69.252.212.76,fus01.ps4.update.playstation.net
36481,1.28197,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,76.120.69.162,69.252.212.72,fus01.ps4.update.playstation.net
49581,1.60432,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,67.177.242.106,69.252.212.68,fus01.ps4.update.playstation.net
55896,1.75579,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,98.245.135.13,69.252.212.88,fus01.ps4.update.playstation.net
72514,2.15917,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,73.95.103.125,69.252.212.68,fus01.ps4.update.playstation.net
73569,2.18296,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,50.155.137.99,69.252.212.68,fus01.ps4.update.playstation.net
85579,2.47275,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,73.95.54.165,69.252.212.78,a01.cdn.update.playstation.net


## XBOX

In [67]:
xbox_df = df[df['dns.qry.name'].str.contains(".xboxlive.com")]

xbox_df


,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
822,0.441398,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,67.177.225.0,69.252.212.78,licensing.xboxlive.com
881,0.442581,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,,,sessiondirectory.xboxlive.com
882,0.442584,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,,,sessiondirectory.xboxlive.com
1065,0.446888,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,,,xsts.auth.xboxlive.com
1066,0.446914,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,,,xsts.auth.xboxlive.com
1580,0.458873,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,67.190.25.80,69.252.212.82,data-vef.xboxlive.com
1586,0.45889,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.86,70.37.134.64,data-vef.gtm.xboxlive.com
1781,0.463833,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,,,licensing.xboxlive.com
2217,0.474166,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,75.71.238.165,69.252.212.78,xsts.auth.xboxlive.com
2318,0.477154,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,71.237.59.204,69.252.212.84,sessiondirectory.xboxlive.com


## NINTENDO

In [68]:
nintendo_df = df[df['dns.qry.name'].str.contains(".nintendo.net")]

nintendo_df


,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name
9074,0.637796,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,75.71.244.2,69.252.212.78,olvus.cdn.nintendo.net
9990,0.659816,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,,,pushmore.wup.shop.nintendo.net
9991,0.659816,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,69.244.53.216,69.252.212.74,pushmore.wup.shop.nintendo.net
10591,0.673846,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,98.245.9.80,69.252.212.78,nppl.app.nintendo.net
12566,0.720612,00:1f:a0:11:b5:3a,40:a8:f0:34:20:2c,71.229.160.157,69.252.212.68,olvus.cdn.nintendo.net
13296,0.737724,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,69.244.53.216,69.252.212.70,nppl.app.nintendo.net
14461,0.766818,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,73.24.228.146,69.252.212.80,tagaya.wup.shop.nintendo.net
14473,0.76707,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.85,192.195.204.190,tagaya.wup.shop.nintendo.net
14681,0.77176,00:1f:a0:11:b5:3a,40:a8:f0:34:35:dc,,,tagaya.wup.shop.nintendo.net
14690,0.771927,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.83,192.195.204.190,tagaya.wup.shop.nintendo.net
